In [3]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pickle as pkl
import tensorflow as tf

import sys
sys.path.append('..')

import metrics

import warnings
warnings.filterwarnings('once')

In [25]:
best_18_model_params = 'inp_18__out_12__loss_mae__bksize_200__bkact_leaky__dir_bi__'
best_12_model_params = 'inp_12__out_12__loss_mae__bksize_100__bkact_leaky__dir_bi__'

p = Path('../results').absolute()

models_dirs_12 = [str(p / best_12_model_params) + str(i) for i in range(10)]
models_dirs_18 = [str(p / best_18_model_params) + str(i) for i in range(10)]

ind = 2

best_12 = models_dirs_12[ind]
best_18 = models_dirs_12[ind]

In [26]:
train_path = Path('../data/Yearly-train.csv')
test_path = Path('../data/Yearly-test.csv')

In [27]:
train = pd.read_csv(train_path).drop('V1', axis=1)
test = pd.read_csv(test_path).drop('V1', axis=1)

In [28]:
np.seterr(all='ignore')

def get_last_N_valid(series, N=18):
    ser_N = series.dropna().iloc[-N:].values
    if len(ser_N) < N:
        return pd.Series([np.nan] * N)
    return ser_N

In [29]:
X_test_18 = np.array([get_last_N_valid(ser[1], N=18) for ser in train.iterrows()])
X_test_12 = np.array([get_last_N_valid(ser[1], N=12) for ser in train.iterrows()])
y_test = test.values

use_12 = np.isnan(X_test_18).any(axis=1)

In [31]:
def get_predictions(model_18, model_12, X_18, X_12, use_12):
    
    preds = []
    
    for i in range(len(use_12)):
        
        if use_12:
            model = model_12
            x = X_12[i]
        else:
            model = model_18
            x = X_18[i]

        mn, mx = x.min(axis=1).reshape(-1, 1), x.max(axis=1).reshape(-1, 1)

        x_sc = (x - mn) / (mx - mn)
        pred = model(x_sc[..., np.newaxis]) 
        preds.append(pred[..., 0] * (mx - mn) + mn)

    return np.vstack(preds)

In [32]:
results = {}

mape = metrics.build_mape(overlap=6)
smape = metrics.build_smape(overlap=6)
mase_estimate = metrics.build_mase(overlap=6)
owa_estimate = metrics.build_owa(overlap=6)
reconstruction_loss = metrics.build_reconstruction_loss(overlap=6)

model_18 = tf.keras.models.load_model(best_18, custom_objects={'SMAPE': smape,
                                                               'MASE_estimate': mase_estimate,
                                                               'OWA_estimate': owa_estimate,
                                                               'reconstruction_loss': reconstruction_loss})

model_12 = tf.keras.models.load_model(best_12, custom_objects={'SMAPE': smape,
                                                               'MASE_estimate': mase_estimate,
                                                               'OWA_estimate': owa_estimate,
                                                               'reconstruction_loss': reconstruction_loss})

preds = get_predictions(model_18, model_12, X_test_18, X_test_12, use_12)

tf.keras.backend.clear_session()

results['smape'] = np.nanmean(metircs.SMAPE(y_test, preds[:, 6:]))
results['mase*'] = np.nanmean(metircs.MASE(y_test, preds[:, 6:]))

OSError: SavedModel file does not exist at: /home/thanos/projects/ae_pred/analysis/../results/inp_12__out_12__loss_mae__bksize_100__bkact_leaky__dir_bi__2/{saved_model.pbtxt|saved_model.pb}